In [2]:
from preproc_tabular import get_tabular
from dataset_sales import getDataset
import sales_prediction as sp
from torch.optim.adamw import AdamW
from torch import nn
import torch
import os

img_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\ResizedImages224\\'
img_path_384 = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\ResizedImages384\\'
desc_path= 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\descrizioni\\descrizioni.pt'
desc_tot_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\descrizioni\\descrizioni_tot.pt'

data_week = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_week.csv'
data_month = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_month.csv'
data_season = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_season.csv'

noneg_week = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_week.csv'
noneg_month = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_month.csv'
noneg_season = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_season.csv'

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Sto usando {device}")

data, references, descriptions, target = get_tabular(noneg_season, desc_tot_path)

modello = sp.create_model(len(data.columns)).to(device)
batch_size = 128
epochs = 40
criterion = nn.MSELoss()
early_stop = sp.EarlyStopping(5, 0.001)

optimizer = AdamW(
    [
        {"params": modello.image.parameters(), "lr": 1e-3},
        {"params": modello.tabular.parameters()},
        {"params": modello.description.parameters(), "lr": 1e-3},
        {"params": modello.final.parameters()}
    ],
    lr=1e-2,
)

train, val = getDataset(references, data, descriptions, target, img_path_384, batch_size, 1)

folder = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\month_nonegozi_prova'
if not os.path.isdir(folder):
    os.makedirs(folder)

mse_final, mae_final, r2_final = 0, 0, 0
min_mae = 100000
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    sp.train_loop(train, modello, criterion, optimizer, batch_size, device)
    mse, mae, r2, bias = sp.validation_loop(val, modello, criterion, device)
    if (min_mae > mae):
        model_sc = torch.jit.script(modello)
        model_sc.save(f'{folder}\\weights.pt')
        min_mae = mae
        print(f'MINIMA MAE AGGIORNATA: {min_mae}')
        
    early_stp = early_stop(mse)
    
    if early_stp:
        print('Early Stop attivato.')
        break
    
print("Fatto!")

Sto usando cuda


c:\Users\GRVRLD00P\Documents\Progetto ORS\Progetto ORS\ProgettoORS\sales_prediction\preproc_tabular.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  descrizioni = torch.l

Epoch 1
-------------------------------


c:\Users\GRVRLD00P\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


MSE: 149183.156250  [  128/ 7998]
In questa epoca, l'MSE medio è 154648.375
Validation Error: 
 Avg MSE: 130666.989294 
 Avg MAE: 250.128287 
 R2: -0.622344
 Bias: -233.097936

MINIMA MAE AGGIORNATA: 250.12828742133246
Epoch 2
-------------------------------
MSE: 128251.601562  [  128/ 7998]
In questa epoca, l'MSE medio è 114716.625
Validation Error: 
 Avg MSE: 85303.288773 
 Avg MAE: 210.162968 
 R2: -0.059549
 Bias: -107.291774

MINIMA MAE AGGIORNATA: 210.16296782316985
Epoch 3
-------------------------------
MSE: 93099.085938  [  128/ 7998]
In questa epoca, l'MSE medio è 83533.5078125
Validation Error: 
 Avg MSE: 74451.823495 
 Avg MAE: 204.053235 
 R2: 0.075720
 Bias: -38.663952

MINIMA MAE AGGIORNATA: 204.0532350893374
Epoch 4
-------------------------------
MSE: 85467.062500  [  128/ 7998]
In questa epoca, l'MSE medio è 75796.7890625
Validation Error: 
 Avg MSE: 68092.255353 
 Avg MAE: 195.794626 
 R2: 0.154835
 Bias: -11.430049

MINIMA MAE AGGIORNATA: 195.79462574146413
Epoch 5


KeyboardInterrupt: 